In [ ]:
from predictor import Predictor

In [ ]:
### Machine learning detection model
predictor = Predictor(config)
os.makedirs('data/predictor', exist_ok=True)
shutil.copyfile(os.path.join('data', 'slides', '4x', acq_name_4x+'.ome.tif'), os.path.join('data/predictor', acq_name_4x+'.ome.tif'))
subprocess.run(["python", "patch_extraction.py", "-s", config["slide-type"], "-f", "tif", "-p", "1"], shell=True)
slides = glob.glob(os.path.join('data-predictor', '*.tif')) + glob.glob(os.path.join('data-predictor', '*.jpg'))
pos = []
image_format = slides[0].split('.')[-1]
for slide in slides:
    slide_name = slide.split(os.sep)[-1].replace('.'+image_format, '')
    fields = glob.glob(os.path.join('data-predictor', slide_name, '*.tif'))
    if config["classifier"] == "Supervised":
        for field in fields:
            pos_x = int(field.split('x=')[1].split(',')[0])
            pos_y = int(field.split('y=')[1].split(',')[0])
            img = io.imread(field)
            prediction = predictor.compute(img)
            c = np.argmax(prediction)
            pos.append([int(pos_x/224), int(pos_y/224), c])
    if config["classifier"] == "MIL":
        prediction_bag, prediction_patch = predictor.compute(fields)
        if prediction_bag:
            for i in range(len(fields)):
                pos_x = int(fields[i].split('x=')[1].split(',')[0])
                pos_y = int(fields[i].split('y=')[1].split(',')[0])
                pos.append([int(pos_x/224), int(pos_y/224), prediction_patch[i]])
pos_arr = np.vstack(pos)
w = np.max(pos_arr, 0)[0]+1
h = np.max(pos_arr, 0)[1]+1
cmap = np.zeros((h ,w))
for p in pos:
    cmap[p[1], p[0]] = p[2]
if config["slide-type"] == "slide":
    cmap_p = morphology.remove_small_objects(img_as_bool(cmap), min_size=6)
    cmap_p = morphology.remove_small_holes(cmap_p, area_threshold=8)
if config["slide-type"] == "TMA":
    cmap_p = cmap
plt.imshow(cmap_p)
os.makedirs(os.path.join('qupath-projects', 'predictions'), exist_ok=True)
io.imsave(os.path.join('qupath-projects', 'predictions', slide_name+'-cmap.bmp'), img_as_ubyte(cmap_p))
# resample 20x bf
field_4x = 224 * config["pixel-size-bf-4x"]
field_w_20x = config["camera-resolution"][0] * config["pixel-size-bf-20x"] * 0.9
field_h_20x = config["camera-resolution"][1] * config["pixel-size-bf-20x"] * 0.9
rw = int(w * (field_4x/field_w_20x))
rh = int(h * (field_4x/field_h_20x))
cmap_r = transform.resize(cmap_p, (rh, rw), order=1, mode='symmetric')
thresh = threshold_mean(cmap_r)*0.5
cmap_r = cmap_r >= thresh
ps_pos = np.where(cmap_r==1)
pos_20x_df = pd.DataFrame({
    'x_pos' : ps_pos[1]*field_w_20x/config["pixel-size-bf-4x"],
    'y_pos' : ps_pos[0]*field_h_20x/config["pixel-size-bf-4x"]}
)
# resample shg
field_mp = 256 * config["pixel-size-shg-base"] * 0.9
rw = int(w * (field_4x/field_mp))
rh = int(h * (field_4x/field_mp))
cmap_r = transform.resize(cmap_p, (rh, rw), order=1, mode='symmetric')
thresh = threshold_mean(cmap_r)*0.5
cmap_r = cmap_r >= thresh
ps_pos = np.where(cmap_r==1)
pos_mp_df = pd.DataFrame({default_bg_20x
    'x_pos' : ps_pos[1]*field_mp/config["pixel-size-bf-4x"],
    'y_pos' : ps_pos[0]*field_mp/config["pixel-size-bf-4x"]}
)
if image_format == 'tif':
    pos_mp_df.to_csv(os.path.join('qupath-projects', 'predictions', 'mp-'+slide_name+'.tif.csv'), index=False)
    pos_mp_df.to_csv(os.path.join('qupath-projects', 'mp-tiles', slide_name+'.tif.csv'), index=False)
    pos_20x_df.to_csv(os.path.join('qupath-projects', 'predictions', '20x-'+slide_name+'.tif.csv'), index=False)
    pos_20x_df.to_csv(os.path.join('qupath-projects', '20x-tiles', slide_name+'.tif.csv'), index=False)
if image_format == 'jpg':
    pos_mp_df.to_csv(os.path.join('qupath-projects', 'predictions', 'mp-'+slide_name+'.csv'), index=False)
    pos_mp_df.to_csv(os.path.join('qupath-projects', 'mp-tiles', slide_name+'.csv'), index=False)
    pos_20x_df.to_csv(os.path.join('qupath-projects', 'predictions', '20x-'+slide_name+'.csv'), index=False)
    pos_20x_df.to_csv(os.path.join('qupath-projects', '20x-tiles', slide_name+'.csv'), index=False)